In [1]:
from selenium import webdriver  # Импортируем библиотеку Selenium для управления веб-браузером
from selenium.webdriver.common.by import By  # Импортируем класс для определения локаторов элементов
from selenium.webdriver.chrome.service import Service  # Импортируем класс для настройки сервиса драйвера Chrome
from selenium.webdriver.support.ui import WebDriverWait  # Импортируем класс для ожидания элементов на странице
from selenium.webdriver.support import expected_conditions as EC  # Импортируем класс для определения условий ожидания
from selenium.common.exceptions import TimeoutException, NoSuchElementException  # Импортируем классы исключений
import pandas as pd  # Импортируем библиотеку для работы с табличными данными

# Настройка драйвера
service = Service(executable_path='C:/WebDriver/bin/chromedriver.exe')  # Указываем путь к драйверу Chrome
driver = webdriver.Chrome(service=service)  # Создаем экземпляр драйвера Chrome

# Базовый URL сайта
base_url = 'https://www.russianfood.com'
url = base_url + "/recipes/bytype/?fid=98"

try:
    recipes = []  # Список для хранения данных о рецептах
    while True:
        driver.get(url)  # Открываем страницу по указанному URL
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".recipe_list_new2")))  # Ожидаем загрузки элемента с классом .recipe_list_new2

        # Собираем данные о рецептах
        recipe_elements = driver.find_elements(By.CSS_SELECTOR, ".in_seen")  # Находим все элементы с классом .in_seen
        for recipe in recipe_elements:
            # Извлекаем название рецепта
            name = recipe.find_element(By.CSS_SELECTOR, "h3[itemprop='name']").text.strip() if recipe.find_elements(By.CSS_SELECTOR, "h3[itemprop='name']") else "Название отсутствует"
            # Извлекаем описание рецепта
            description = recipe.find_element(By.CSS_SELECTOR, "[itemprop='description']").text.strip() if recipe.find_elements(By.CSS_SELECTOR, "[itemprop='description']") else "Описание отсутствует"
            # Извлекаем ингредиенты рецепта
            ingredients = recipe.find_element(By.CSS_SELECTOR, ".announce_sub span").text.strip() if recipe.find_elements(By.CSS_SELECTOR, ".announce_sub span") else "Ингредиенты отсутствуют"
            
            # Убираем слово "Продукты:" из ингредиентов
            if ingredients.startswith("Продукты:"):
                ingredients = ingredients.replace("Продукты:", "").strip()
            
            # Добавляем данные рецепта в список
            recipes.append({'name': name, 'description': description, 'ingredients': ingredients})
            
            if len(recipes) >= 50:  # Ограничение количества рецептов
                raise Exception("Limit reached")  # Вызов исключения для выхода из цикла

        # Ищем кнопку следующей страницы и переходим по ссылке
        next_page = driver.find_element(By.CSS_SELECTOR, "a[href*='page=']")  # Находим ссылку на следующую страницу
        if next_page:
            url = base_url + next_page.get_attribute('href')  # Обновляем URL для перехода на следующую страницу
        else:
            break  # Если кнопка следующей страницы отсутствует, завершаем цикл

except (TimeoutException, NoSuchElementException):
    print("Не удалось загрузить страницу или достигнут конец списка.")  # Обрабатываем исключения на случай, если страница не загрузилась или достигнут конец списка
except Exception as e:
    print(str(e))  # Выводим сообщение об ошибке

driver.quit()  # Закрываем браузер

# Создаем DataFrame из собранных данных
df = pd.DataFrame(recipes)
if not df.empty:
    df.to_csv('recipes_50.csv', index=False)  # Сохраняем данные в CSV файл
    print("Данные сохранены в файл 'recipes_50.csv'")  # Сообщаем об успешном сохранении данных
else:
    print("Нет данных для сохранения.")  # Сообщаем, если нет данных для сохранения


Limit reached
Данные сохранены в файл 'recipes_50.csv'
